Solution to the following programming question:

Using sklearn.model_selection.permutation_test_score, compute the p-value indicating if the score obtained by an instance of sklearn.dummy.DummyClassifier on the dataset sklearn.datasets.load_iris is obtained by chance.

Repeat this analysis using sklearn.ensemble.HistGradientBoostingClassifier instead of sklearn.dummy.DummyClassifier.

What can you conclude about:

the existence of a significant statistical association between the iris type and the input features (petal and sepal width and length)?
the ability of each kind of estimator to assess or not such a statistical association between features and target variable?


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np
from sklearn import datasets
import scipy.stats as stats
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

In [ ]:
def is_obtained_by_chance(nr_classes: int, nr_samples: int, score: float,
                          alpha: float = .05) -> bool:
    """Check if the submitted score is just as good or bad as random guessing.

    In case of random guessing the probability of obtaining the correct output
    per sample equals 1/nr_classes, thus the proportion of correctly classified
    samples (p0) would equal 1/nr_classes.
    Because of the law of large number we know that for a large number of
    samples the sampling distribution of the sample proportion follows a
    normal distribution. However, since the variance must be estimated,
    we assume a t-distribution.
    Therefore the following hypothesis test (t-test) is conducted:

    Hypothesis 0: proportion = 1/nr_classes
    Hypothesis A: proportion != 1/nr_classes

    Thus, under the assumption that the Null hypothesis is correct,
    the test-statistic is a t-distribution with mean p0, with standard
    deviation equal to the standard error of the mean (sem)
    (sample variance/sqrt(nr_samples)) and with degrees of freedom equal to
    nr_samples -1.

    Parameters
    ----------
    nr_classes : int
        number of possible output classes for the given classification task
    nr_samples : int
        number of samples that the score was obtained from
    score : float
       proportion of correctly classified samples
    alpha : float, default = 0.05
      -desired significance-level for the test, i.e. 1-confidence-level

    Returns
    -------
    obtained_by_chance : bool
        True, if the model performance is comparable to random guessing, i.e. H0 was not rejected
        else False
    """
    obtained_by_chance = True

    p0 = 1/nr_classes
    sem = np.sqrt(p0 * (1-p0) / nr_samples)
    p_value = stats.t(df=nr_samples-1, loc=p0, scale=sem).cdf(score)
    # model performs better than chance -> reject H0
    if p_value < alpha/2:
        obtained_by_chance = False
        return obtained_by_chance
    # model performs worse than chance -> reject H0
    if (1-p_value) < alpha/2:
        obtained_by_chance = False
        return obtained_by_chance

    return obtained_by_chance

In [ ]:
# load the data, train model on training set and apply to test set
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,
    random_state=0)

dummy_clf = DummyClassifier()
dummy_clf.fit(X_train, y_train)
score = dummy_clf.score(X_test, y_test)
print('The assumption that the DummyClassifier performance is not different from random guessing is',
      is_obtained_by_chance(3, len(y_test), score))

# now repeat with a different classifier
model = HistGradientBoostingClassifier(max_bins=255, max_iter=100)
model.fit(X_train, y_train)
prediction_boosting = model.predict(X_test)
score = model.score(X_test, y_test)
print('The assumption that the HistGradientBoostingClassifier performance is not different from random guessing is',
      is_obtained_by_chance(3, len(y_test), score))
